In [32]:
import numpy as np
import asyncio

# Create a fake trade feed

In [75]:
async def trade_feed(num_trade=10):
    price = 100
    for _ in range(num_trade):
        price *= np.exp(np.random.normal(scale=0.01))
        price = float(np.round(price, 2))
        volume = int(np.random.normal(scale=100))
        yield volume, price
        dt = np.random.uniform(0, 0.2)
        await asyncio.sleep(dt)

async for v, p in trade_feed():
    print(v, p)
    

-112 100.51
2 102.17
23 101.31
24 101.39
-150 100.22
81 101.4
-197 102.11
135 101.51
-165 100.38
50 99.71


### A simple Cashflow class: compute price * volume

In [82]:
from collections.abc import Iterable
import inspect

class Cashflow:

    def __init__(self):
        pass

    # main implementation
    def process_scalar(self, volume, price):
        return abs(volume * price)

    # generic code: handeling generator of tuples
    def process_generator(self, values_gen):
        for values in values_gen:
            yield self.process_scalar(*values)

    # generic code: handeling numpy arrays
    def process_numpy_arrays(self, volumes, prices):
        ans = np.zeros_like(prices)
        for i in range(len(prices)):
            ans[i] = self.process_scalar(volumes[i], prices[i])
        return ans

    # generic code: handeling an asynchronous generator of tuples
    async def process_async_generator(self, async_values_gen):
        async for values in async_values_gen:
            yield self.process_scalar(*values)  
        
    # generic code: dispatcher
    def __call__(self, arg0, arg1=None):
        if isinstance(arg0, np.ndarray):
            return self.process_numpy_arrays(arg0, arg1)
        if isinstance(arg0, Iterable):
            return self.process_generator(arg0)
        if inspect.isasyncgen(arg0):
            return self.process_async_generator(arg0)
        return self.process_scalar(arg0, arg1)


### A Scalar

In [83]:
cf = Cashflow()

cf(3,4)

12

### Numpy arrays

In [84]:
prices = np.array([100,101,102,101])
volumes = np.array([3,-1,-2,1])

cf = Cashflow()
cf(volumes, prices)


array([300, 101, 204, 101])

### Generator

In [85]:
gen = iter([(2,300), (-1, 101), (-2, 102), (1, 101)])


list(cf(gen))

[600, 101, 204, 101]

### Async generator

In [86]:
cf = Cashflow()

async for c in cf(trade_feed()):
    print(c)


8947.17
7129.82
4377.4
1622.72
20653.75
5799.179999999999
12781.44
11806.47
2379.84
5520.9
